This notebook uploads the csv from **filter.ipynb** to Argilla to filter non-relevant samples manually (from a financial advisor pov).

In [1]:
import os
import argilla as rg
rg.init(api_url=os.environ["ARGILLA_URL"], api_key=os.environ["ARGILLA_API_KEY"])
rg.set_workspace(os.environ["ARGILLA_WORKSPACE"])

/Users/pablo/.local/share/virtualenvs/finadvice-dataset-mdWLTC9R/lib/python3.11/site-packages/argilla/client/client.py:152: UserWarning: You're connecting to Argilla Server 1.13.3 using a different client version (1.14.1).
This may lead to potential compatibility issues during your experience.
To ensure a seamless and optimized connection, we highly recommend aligning your client version with the server version.
  warnings.warn(


In [39]:
"""
schema = {
    "created_utc": pl.Int32,
    "id": pl.Utf8,
    "is_self": pl.Boolean,
    "title": pl.Utf8,
    "selftext": pl.Utf8,
    "subreddit": pl.Utf8,
}
"""

import pandas as pd
df = pd.read_csv("./outdata/submissions.csv")
display(df.sample())
len(df)

,id,title,created_utc,edited,selftext
52274,u7z786,Should I downgrade my car to have money to inv...,1650468800,1.650471e+09,Context: I’m currently only 17 and am about to...


84944

In [40]:
df = df[df["edited"] == -1]
len(df)

73536

In [46]:
records = []

for idx, row in df[:1000].iterrows():
    metadata = {
        "created_utc": row["created_utc"],
        "id": row["id"],
        "title": row["title"],
    }

    text = row["selftext"]

    records.append(
        rg.TextClassificationRecord(
            text=text,
            metadata=metadata,
            multi_label=True,
        )
    )

/var/folders/7d/4d1w6fjj4fgddrc8vppvmyv80000gn/T/ipykernel_74487/298426010.py:13: UserWarning: Some metadata values could exceed the max length. For those cases, values will be truncated by keeping only the last 128 characters. You can configure this length in the server with the ARGILLA_METADATA_FIELD_LENGTH environment variable. Note that, setting this too high may lead to Elastic performance issues.
  rg.TextClassificationRecord(


In [47]:
len(records)

1000

In [48]:
rg.set_workspace("shared")
rg.log(records=records, name="fin_advice_relevant_1000_2022_4", batch_size=100)

/Users/pablo/.local/share/virtualenvs/finadvice-dataset-mdWLTC9R/lib/python3.11/site-packages/rich/live.py:231: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

1000 records logged to 
]8;id=483407;https://21uz3hkjad87iu41kj3heo982idak3i1378814oijda9s.nihiluis.com/datasets/shared/fin_advice_relevant_1000_2022_4\https://21uz3hkjad87iu41kj3heo982idak3i1378814oijda9s.nihiluis.com/datasets/shared/fin_advice_relevant_1000_2022_4]8;;\

BulkResponse(dataset='fin_advice_relevant_1000_2022_4', processed=1000, failed=0)